In [ ]:
from scipy.io import loadmat
import pandas as pd

import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import datetime
import matplotlib.pyplot as plt

In [ ]:
mat = loadmat('data/letter.mat')
# mat = loadmat('data/satimage-2.mat')
# mat = loadmat('data/cardio.mat')

df = pd.DataFrame(mat['X'])
df['y'] = mat['y']

X_train, X_test = train_test_split(df, test_size=0.2)
X_train = X_train[X_train['y'] == 0]
X_train.drop(['y'], axis=1)

y_test = X_test['y']
X_test.drop(['y'], axis=1)

X_train = X_train.values
X_test = X_test.values

print('Training data size:  ', X_train.shape)
print('Validation data size:', X_test.shape)

input_dim = X_train.shape[1]

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# input_layer = Input(shape=(input_dim,))
# encoder = Dense(20, activation='tanh')(input_layer)
# encoder = Dense(10, activation='tanh')(encoder)
# encoder = Dense(5, activation='tanh')(encoder)
# decoder = Dense(10, activation='tanh')(encoder)
# decoder = Dense(20, activation='tanh')(decoder)
# decoder = Dense(input_dim, activation='sigmoid')(decoder)
# autoencoder = Model(inputs=input_layer, outputs=decoder)
# autoencoder.summary()

In [ ]:
def create_undercomplete_autoencoder(n_hidden, activation='relu', summary=False):
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(n_hidden, activation=activation)(input_layer)
    decoder = Dense(input_dim, activation='sigmoid')(encoder)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    if summary:
        autoencoder.summary()

    return autoencoder

def create_sparse_autoencoder(n_hidden, activation='relu', summary=False):
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(n_hidden, activation=activation, activity_regularizer=regularizers.l1(10e-5))(input_layer)
    decoder = Dense(input_dim, activation='sigmoid')(encoder)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    if summary:
        autoencoder.summary()

    return autoencoder

In [ ]:
def train(autoencoder, data, epochs=100, batch_size=50, verbose=False):
    autoencoder.compile(optimizer='adam', loss='mse')
    history = autoencoder.fit(
        data,
        data,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=True,
        validation_split=0.1,
        verbose=verbose
    )

In [ ]:
def describe(autoencoder, data_X, data_Y, describe=False, plot=False):
    predictions = autoencoder.predict(data_X)

    mse = np.mean(np.power(data_X - predictions, 2), axis=1)
    df_error = pd.DataFrame({'reconstruction_error': mse, 'Label': data_Y}, index=data_Y.index)
    if describe:
        df_error.describe()
    if plot:
        plt.figure()
        plt.scatter(df_error.index, df_error['reconstruction_error'].values)
        plt.show()

    return df_error

In [ ]:
def evaluate(df_error, data_Y, plot=False):
    rec_error = df_error['reconstruction_error']
    result = {}
    min_threshold = 0
    min_number_of_errors = data_Y.size
    outliers = data_Y.index.values
    for outlier in outliers:
        threshold = rec_error[outlier]
        number_of_errors = 0
        for key in rec_error.index:
            value = rec_error[key]
            y = data_Y[key]
            classification = value >= threshold
            if classification != y:
                number_of_errors += 1

        result[threshold] = number_of_errors

        if number_of_errors < min_number_of_errors:
            min_number_of_errors = number_of_errors
            min_threshold = threshold

    if plot:
        plt.figure()
        plt.scatter(result.keys(), result.values())
        plt.show()

    return (min_threshold, min_number_of_errors)

In [ ]:
def plot_results(x, number_of_errors, thresholds):
    data1 = np.array(number_of_errors)
    data2 = np.array(thresholds)

    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('Number of hidden nodes')
    ax1.set_ylabel('Number of misclassified points', color=color)
    ax1.plot(x, data1, color=color, marker='o')
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()

    color = 'tab:blue'
    ax2.set_ylabel('Threshold', color=color)
    ax2.plot(x, data2, color=color, marker='o')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()
    plt.show()

In [ ]:
thresholds = []
number_of_errors = []
for i in range(1, input_dim):
    print("Number of hidden nodes: " + str(i))
    model = create_undercomplete_autoencoder(n_hidden=i)
    train(model, X_train_scaled)
    df_error = describe(model, X_test_scaled, y_test, plot=True)
    t, e = evaluate(df_error, y_test)
    thresholds.append(t)
    number_of_errors.append(e)

In [ ]:
plot_results(range(1, input_dim), number_of_errors, thresholds)

In [ ]:
thresholds = []
number_of_errors = []
for i in range(1, input_dim):
    print("Number of hidden nodes: " + str(i))
    model = create_sparse_autoencoder(n_hidden=i)
    train(model, X_train_scaled)
    df_error = describe(model, X_test_scaled, y_test, plot=True)
    t, e = evaluate(df_error, y_test)
    thresholds.append(t)
    number_of_errors.append(e)

In [ ]:
plot_results(range(1, input_dim), number_of_errors, thresholds)

In [ ]:
functions = ['relu', 'sigmoid', 'tanh', 'softmax']

thresholds = []
number_of_errors = []
for f in functions:
    print("Activation: " + f)
    model = create_undercomplete_autoencoder(n_hidden=6, activation=f)
    train(model, X_train_scaled)
    df_error = describe(model, X_test_scaled, y_test, plot=True)
    t, e = evaluate(df_error, y_test)
    thresholds.append(t)
    number_of_errors.append(e)
plot_results(functions, number_of_errors, thresholds)

thresholds = []
number_of_errors = []
for f in functions:
    print("Activation: " + f)
    model = create_sparse_autoencoder(n_hidden=6, activation=f)
    train(model, X_train_scaled)
    df_error = describe(model, X_test_scaled, y_test, plot=True)
    t, e = evaluate(df_error, y_test)
    thresholds.append(t)
    number_of_errors.append(e)
plot_results(functions, number_of_errors, thresholds)
